**OVERVIEW**

In this project, we will clean and analyze data from the spotify-2023.csv, and spotify-2024.csv files.

The project has 3 goals:
1. Idenitify the most-common musical attribute values of all 2023 Spotify songs.
2. Count the occurances of the most-common attribute values within the entire spotify-2023.csv file, as well as within only the top-10 most streamed songs of 2023.
3. Merge the two csv files, and track the top-10 songs' stream count growth from 2023-2024.

In [688]:
import pandas as pd
from fuzzywuzzy import process

**Cleaning 1: spotify-2023.csv Columns and Rows**

1. The Spotify-2023.csv file is cleaned below by reading only the columns needed. The 'track_name' and 'artist(s)_name' columns identify the song and artist, the 'streams' column identifies the popularity of the song, and the 'bpm', 'key', and 'mode' columns identify the song's musical attributes.

2. Rename the columns to match the spotify-2024.csv. This will help when we merge the two csv files later in the project.

3. Convert the 'streams' column to numeric using to_numeric() forcing errors to NaN.

4. Drop rows with NaN using .dropna().

5. Convert 'streams' column to integers using .astype(int).

6. Eliminate white space, and convert to lowercase by using .strip(), and .lower(). 

7. Sort the DataFrame by 'streams' column in descending order using .sort() to have the most popular songs at the top of the DataFrame.

In [689]:
columns_to_read = ['track_name', 'artist(s)_name', 'streams', 'bpm', 'key', 'mode']
df_2023 = pd.read_csv('spotify-2023.csv', usecols= columns_to_read, encoding= 'latin-1')

# Rename columns to match 2024 file.
df_2023 = df_2023.rename(columns={
    'track_name': 'Track',
    'artist(s)_name': 'Artist',
    'streams': 'Streams_2023' 
})

# Convert 'Streams_2023' column to numeric, forcing errors to NaN
df_2023['Streams_2023'] = pd.to_numeric(df_2023['Streams_2023'], errors='coerce')

# Drop rows with NaN if necessary
df_2023 = df_2023.dropna()

# Convert 'Streams_2023' column to integers
df_2023['Streams_2023'] = df_2023['Streams_2023'].astype(int)

# Eliminate white space and convert to lowercase
df_2023['Track'] = df_2023['Track'].str.replace(r'\s+', ' ', regex=True).str.strip().str.lower()
df_2023['Artist'] = df_2023['Artist'].str.replace(r'\s+', ' ', regex=True).str.strip().str.lower()

# Sort the DataFrame by 'Streams_2023' column in descending order
df_2023_desc = df_2023.sort_values(by='Streams_2023', ascending=False)

print(df_2023_desc)

                                             Track  \
55                                 blinding lights   
179                                   shape of you   
86                               someone you loved   
620                                   dance monkey   
41   sunflower - spider-man: into the spider-verse   
..                                             ...   
30                                            rush   
68                                       overdrive   
142                    gol bolinha, gol quadrado 2   
393                              jhoome jo pathaan   
123                                    que vuelvas   

                                                Artist  Streams_2023  bpm key  \
55                                          the weeknd    3703895074  171  C#   
179                                         ed sheeran    3562543890   96  C#   
86                                       lewis capaldi    2887241814  110  C#   
620                        

**Analysis 1: Most Common Attributes**

Rather than tracking the songs simply based on popularity alone, we will also take a look at each song's musical attributes.

Now that we have a DataFrame of cleaned data, we need to idenitify the most-common musical attributes for each song in the entire DataFrame.

1. Find the most-common BPM, KEY, and MODE used for each song by using .mode().

2. Once the attribute values are identified, create a dictionary to store those values so we can convert them into a DataFrame to print the findings.

In [690]:
common_bpm = df_2023_desc['bpm'].mode()[0]
common_key = df_2023_desc['key'].mode()[0]
common_mode = df_2023_desc['mode'].mode()[0]


# Create a dictionary with these values
common_attributes_dict = {
    'Musical Attribute': ['BPM', 'Key', 'Mode'],
    'Most Common Value Among 2023 Spotify Songs': [common_bpm, common_key, common_mode]
}
# Convert the dictionary to a DataFrame
df_common_attributes = pd.DataFrame(common_attributes_dict)

print('2023 Musical Attribute Values:')
print(df_common_attributes.to_string(index=False))

print()
print(common_bpm, 'is the most common BPM of 2023 Spotify songs.')
print(common_key, 'is the most common KEY of 2023 Spotify songs.')
print(common_mode, 'is the most common MODE of 2023 Spotify songs.')


2023 Musical Attribute Values:
Musical Attribute Most Common Value Among 2023 Spotify Songs
              BPM                                        120
              Key                                         C#
             Mode                                      Major

120 is the most common BPM of 2023 Spotify songs.
C# is the most common KEY of 2023 Spotify songs.
Major is the most common MODE of 2023 Spotify songs.


**Analysis 2: Statistics of Attribute Values**

Now, we will calculate *how often* the the most-common attribute values occur in the entire DataFrame. This will take 4 steps.

1. Count how many times an attribute value occures in the column using .value_counts().

2. Count the total number of entries for the column using .count().

3. Divide the attribute value count and total count, then multiply by 100 to get the percentage.

4. Round the percentage to the nearest 2 decimal points using the round() function.

Once again, we will store the findings in a dictionary so we can convert them into a DataFrame to be printed.

In [691]:
#BPM Statistics
count_bpm = df_2023_desc['bpm'].value_counts()[common_bpm]
total_bpm = df_2023_desc['bpm'].count()
percentage_bpm = (count_bpm / total_bpm) * 100
percentage_bpm = round(percentage_bpm, 2)

#KEY Statistics
count_key = df_2023_desc['key'].value_counts()[common_key]
total_key = df_2023_desc['key'].count()
percentage_key = (count_key / total_key) * 100
percentage_key = round(percentage_key, 2)

#MODE Statistics
count_mode = df_2023_desc['mode'].value_counts()[common_mode]
total_mode = df_2023_desc['mode'].count()
percentage_mode = (count_mode / total_mode) * 100
percentage_mode = round(percentage_mode, 2)

# Create a dictionary with these values
attribute_count_dict = {
    'Musical Attribute': ['BPM', 'Key', 'Mode'],
    'Most Common Value': [common_bpm, common_key, common_mode],
    'Attribute Value Count': [count_bpm, count_key, count_mode],
    'Attribute Value Percentage': [percentage_bpm, percentage_key, percentage_mode]
}
# Convert the dictionary to a DataFrame
df_attribute_count = pd.DataFrame(attribute_count_dict)

print('2023 Attribute Value Count:')
print(df_attribute_count.to_string(index=False))

print()
print(common_bpm, 'is the most common BPM of 2023 Spotify songs. It occurs', count_bpm, 'times.', 'This is', percentage_bpm, 'percent of 2023 Spotify songs.')
print()
print(common_key, 'is the most common KEY of 2023 Spotify songs. It occurs', count_key, 'times.', 'This is', percentage_key, 'percent of 2023 Spotify songs.')
print()
print(common_mode, 'is the most common MODE of 2023 Spotify songs. It occurs', count_mode, 'times.', 'This is', percentage_mode, 'percent of 2023 Spotify songs.')

2023 Attribute Value Count:
Musical Attribute Most Common Value  Attribute Value Count  Attribute Value Percentage
              BPM               120                     34                        3.97
              Key                C#                    120                       14.00
             Mode             Major                    474                       55.31

120 is the most common BPM of 2023 Spotify songs. It occurs 34 times. This is 3.97 percent of 2023 Spotify songs.

C# is the most common KEY of 2023 Spotify songs. It occurs 120 times. This is 14.0 percent of 2023 Spotify songs.

Major is the most common MODE of 2023 Spotify songs. It occurs 474 times. This is 55.31 percent of 2023 Spotify songs.


**Now let's take a look at the top-10 streamed songs of the DataFrame**

In [692]:
df_top10 = df_2023_desc.head(10)

print(df_top10)

                                             Track  \
55                                 blinding lights   
179                                   shape of you   
86                               someone you loved   
620                                   dance monkey   
41   sunflower - spider-man: into the spider-verse   
162                                      one dance   
84                       stay (with justin bieber)   
140                                       believer   
725                                         closer   
48                                         starboy   

                           Artist  Streams_2023  bpm key   mode  
55                     the weeknd    3703895074  171  C#  Major  
179                    ed sheeran    3562543890   96  C#  Minor  
86                  lewis capaldi    2887241814  110  C#  Major  
620                   tones and i    2864791672   98  F#  Minor  
41          post malone, swae lee    2808096550   90   D  Major  
162      

**Analysis 3: Statistics of Attribute Values in Top-10 songs**

Now, we will calculate how often the the most-common attribute values occur in only the top 10 songs.

The calculations above in Analysis 2 might work, but can raise errors if those attribute values don't appear in the DataFrame. We will add some code to prevent errors, specifically in the BPM statistics where the most common BPM does not appear in the top 10.

1. Count how many times an attribute occures in the column ***using .get().*** This will default to 0 if the value you are trying to find is not present (which is the case with BPM.)

2. Count the total number of entries for the column using .count().

3. Divide the attribute count and total count, then multiply by 100 to get the percentage.

4. Round the percentage to the nearest 2 decimal points using the round() function.

Once again, we will store the findings in a dictionary so we can convert them into a DataFrame to be printed.

In [693]:
#BPM Statistics in the Top 10 songs
count_bpm_top10 = df_top10['bpm'].value_counts().get(common_bpm, 0) #Get count, or default to 0 if necessary
total_bpm_top10 = df_top10['bpm'].count()
percentage_bpm_top10 = (count_bpm_top10 / total_bpm_top10) * 100
percentage_bpm_top10 = round(percentage_bpm_top10, 2)

#Key Statistics in the Top 10 songs
count_key_top10 = df_top10['key'].value_counts().get(common_key, 0) #Get count, or default to 0 if necessary
total_key_top10 = df_top10['key'].count()
percentage_key_top10 = (count_key_top10 / total_key_top10) * 100
percentage_key_top10 = round(percentage_key_top10, 2)

#Mode Statistics in the Top 10 songs
count_mode_top10 = df_top10['mode'].value_counts().get(common_mode, 0) #Get count, or default to 0 if necessary
total_mode_top10 = df_top10['mode'].count()
percentage_mode_top10 = (count_mode_top10 / total_mode_top10) * 100
percentage_mode_top10 = round(percentage_mode_top10, 2)

# Create a dictionary with these values
top10_count_dict = {
    'Musical Attribute': ['BPM', 'Key', 'Mode'],
    'Most Common Value': [common_bpm, common_key, common_mode],
    'Top 10 Count': [count_bpm_top10, count_key_top10, count_mode_top10],
    'Top 10 Percentage': [percentage_bpm_top10, percentage_key_top10, percentage_mode_top10],
}

# Convert the dictionary to a DataFrame
df_attribute_count_top10 = pd.DataFrame(top10_count_dict)

print('Attribute count in the top-10 2023 Spotify songs:')
print(df_attribute_count_top10.to_string(index=False))

print()
print(common_bpm, 'BPM occurs', count_bpm_top10, 'times in the top-10 2023 Spotify songs. This is', percentage_bpm_top10, 'percent of the top-10 songs, as opposed to', percentage_bpm, 'percent of all 2023 Spotify songs.')
print()
print(common_key, ' KEY occurs', count_key_top10, 'times in the top-10 2023 songs. This is', percentage_key_top10, 'percent of the top-10 songs, as opposed to', percentage_key, 'percent of all 2023 Spotify songs.')
print()
print(common_mode, 'MODE occurs', count_mode_top10, 'times in the top-10 2023 songs. This is', percentage_mode_top10, 'percent of the top-10 songs, as opposed to', percentage_mode, 'percent of all 2023 Spotify songs.')


Attribute count in the top-10 2023 Spotify songs:
Musical Attribute Most Common Value  Top 10 Count  Top 10 Percentage
              BPM               120             0                0.0
              Key                C#             5               50.0
             Mode             Major             7               70.0

120 BPM occurs 0 times in the top-10 2023 Spotify songs. This is 0.0 percent of the top-10 songs, as opposed to 3.97 percent of all 2023 Spotify songs.

C#  KEY occurs 5 times in the top-10 2023 songs. This is 50.0 percent of the top-10 songs, as opposed to 14.0 percent of all 2023 Spotify songs.

Major MODE occurs 7 times in the top-10 2023 songs. This is 70.0 percent of the top-10 songs, as opposed to 55.31 percent of all 2023 Spotify songs.


**Cleaning 2: Spotify-2024.csv**

We have cleaned and analyzed the spotify-2023.csv file. The next part of this project will look at the top-ten 2023 songs, and compare their 'streams' count from 2023 to their 2024 current count. 

First, we will read and clean the spotify-2024.csv file. We can use mostly the same method as we did for the 2023.csv file. However, we need to add two steps. The 2024.csv 'Spotify Streams' contains commas, which results in errors. The new steps to clean the file are as follows:

1. Read only the columns needed: 'Track', 'Artist', 'Spotify Streams'.

2. Rename 'Spotify Streams' column for easier comparison to 2023 streams.

3. **NEW** Convert 'Spotify Streams' to a string using .astype(str).

4. **NEW** Check for commas, and replace (if present) using str.replace.

5. Convert the 'streams' column to numeric using to_numeric() forcing errors to NaN.

6. Drop rows with NaN using .dropna().

7. Convert 'Spotify Streams' column to integers using .astype(int).

8. Eliminate white space, and convert to lowercase by using .strip(), and .lower(). 

In [694]:

columns_to_read = ['Track', 'Artist', 'Spotify Streams']
df_2024 = pd.read_csv('spotify-2024.csv', usecols=columns_to_read, encoding='latin-1')


# Rename Spotify Streams column
df_2024 = df_2024.rename(columns={
    'Spotify Streams': 'Streams_2024'
})

# NEW: Convert to string for inspection
df_2024['Streams_2024'] = df_2024['Streams_2024'].astype(str)

# NEW: Check for commas
df_2024['Streams_2024'] = df_2024['Streams_2024'].str.replace(',', '')  #<-- Remove commas if present

#Convert 'Streams_2024' to numeric, forcing errors to NaN
df_2024['Streams_2024'] = pd.to_numeric(df_2024['Streams_2024'], errors='coerce')

# Drop rows with NaN values'
df_2024 = df_2024.dropna()

# Convert 'Streams_2024' column to integers
df_2024['Streams_2024'] = df_2024['Streams_2024'].astype(int)


# Lowercase and eliminate white space
df_2024['Track'] = df_2024['Track'].str.replace(r'\s+', ' ', regex=True).str.strip().str.lower()
df_2024['Artist'] = df_2024['Artist'].str.replace(r'\s+', ' ', regex=True).str.strip().str.lower()


print(df_2024)

                                Track          Artist  Streams_2024
0                 million dollar baby   tommy richman     390470936
1                         not like us  kendrick lamar     323703884
2          i like the way you kiss me         artemas     601309283
3                             flowers     miley cyrus    2031280633
4                             houdini          eminem     107034922
...                               ...             ...           ...
4595                for the last time     $uicideboy$     305049963
4596                 dil meri na sune      atif aslam      52282360
4597            grace (feat. 42 dugg)        lil baby     189972685
4598              nashe si chadh gayi    arijit singh     145467020
4599  me acostumbre (feat. bad bunny)       arcï¿½ï¿½     255740653

[4487 rows x 3 columns]


**Analysis 4: MERGE df_top10 and df_2024 DataFrames**

Now that both DataFrames are cleaned, we will use pd.merge to combine them. 

**Note** The streams_2024 column outputs scientific notation due to the value being so large. We will convert the column to an interger to prevent this.

In [695]:
# Merge the top ten songs from 2023 with the 2024 data
df_comparison = pd.merge(df_top10, df_2024, on=['Track', 'Artist'], how='left')

# Convert Streams_2024 to integer and handle scientific notation
df_comparison['Streams_2024'] = df_comparison['Streams_2024'].fillna(0).astype(int)

print(df_comparison)

                                           Track  \
0                                blinding lights   
1                                   shape of you   
2                              someone you loved   
3                                   dance monkey   
4  sunflower - spider-man: into the spider-verse   
5                                      one dance   
6                      stay (with justin bieber)   
7                                       believer   
8                                         closer   
9                                        starboy   

                         Artist  Streams_2023  bpm key   mode  Streams_2024  
0                    the weeknd    3703895074  171  C#  Major    4281468720  
1                    ed sheeran    3562543890   96  C#  Minor    3909458734  
2                 lewis capaldi    2887241814  110  C#  Major    3427498835  
3                   tones and i    2864791672   98  F#  Minor    3071214106  
4         post malone, swae lee    28

**Analysis 5: Fuzzy Matching**

We see that the merged DataFrames are missing values in the Streams_2024 column. This is due to discrepencies in the Artist columns between the two original csv files. We will use Fuzzy Matching, which can handle misspellings, abbreviations, and slight variations in artist names. 

1. Define the standard artist names as a list artist_list. 

2. Define fuzzy matching function. This takes an artist's name as input, uses fuzzy matching to find the closest match from artist_list.

3. Creat sample DataFrames.

4. Standardize Artist names. The fuzzy function is applied to the Artist column in both DataFrames. This step ensures that artist names are standardized according to the artist_list, making it easier to compare data across years.

5. Merge DataFrames. Use pd.merge to merge the DFs on the 'Track' column.

6. Select columns and rename for consistency. The column Artist_2023 is renamed to Artist to make it consistent across the DataFrame.

7. Drop duplicates using .drop_duplicates().

In [696]:
# Define standard artist names
artist_list = [
    'post malone, swae lee',
    'drake, wizkid, kyla',
    'justin bieber, the kid laroi',
    'the chainsmokers, halsey',
    'the weeknd',
    'the weeknd, daft punk'
]

# Fuzzy Matching function
def fuzzy_standardize_artist_names(artist_name):
    best_match = process.extractOne(artist_name, artist_list)
    if best_match and best_match[1] >= 80:  # Adjust threshold if needed
        return best_match[0]
    return artist_name

# Sample data for testing
df_2023 = pd.DataFrame({
    'Track': ['blinding lights', 'shape of you', 'someone you loved', 'dance monkey', 'sunflower - spider-man: into the spider-verse', 'one dance', 'stay (with justin bieber)', 'stay (with justin bieber)', 'believer', 'closer', 'starboy'],
    'Artist': ['the weeknd', 'ed sheeran', 'lewis capaldi', 'tones and i', 'post malone', 'drake', 'justin bieber, the kid laroi', 'justin bieber, the kid laroi', 'imagine dragons', 'the chainsmokers, halsey', 'the weeknd, daft punk'],
    'Streams_2023': [3703895074, 3562543890, 2887241814, 2864791672, 2808096550, 2713922350, 2665343922, 2665343922, 2594040133, 2591224264, 2565529693]
})

df_2024 = pd.DataFrame({
    'Track': ['blinding lights', 'shape of you', 'someone you loved', 'dance monkey', 'sunflower - spider-man: into the spider-verse', 'one dance', 'stay (with justin bieber)', 'stay (with justin bieber)', 'believer', 'closer', 'starboy'],
    'Artist': ['the weeknd', 'ed sheeran', 'lewis capaldi', 'tones and i', 'post malone, swae lee', 'drake, wizkid, kyla', 'justin bieber, the kid laroi', 'justin bieber, the kid laroi', 'imagine dragons', 'the chainsmokers, halsey', 'the weeknd, daft punk'],
    'Streams_2024': [4281468720, 3909458734, 3427498835, 3071214106, 3358704125, 3192204066, 3107100349, 3107100349, 3006226762, 2880536961, 3291262413]
})


# Apply the function to standardize artist names
df_2023['Artist'] = df_2023['Artist'].apply(fuzzy_standardize_artist_names)
df_2024['Artist'] = df_2024['Artist'].apply(fuzzy_standardize_artist_names)

# Merge DataFrames on 'Track'
df_comparison = pd.merge(df_2023, df_2024, on='Track', suffixes=('_2023', '_2024'))

# Select relevant columns and rename for consistency
df_comparison = df_comparison[['Track', 'Artist_2023', 'Streams_2023', 'Streams_2024']]
df_comparison.rename(columns={'Artist_2023': 'Artist'}, inplace=True)


# Drop duplicates if any
df_comparison = df_comparison.drop_duplicates()

print(df_comparison)

                                            Track  \
0                                 blinding lights   
1                                    shape of you   
2                               someone you loved   
3                                    dance monkey   
4   sunflower - spider-man: into the spider-verse   
5                                       one dance   
6                       stay (with justin bieber)   
10                                       believer   
11                                         closer   
12                                        starboy   

                          Artist  Streams_2023  Streams_2024  
0                     the weeknd    3703895074    4281468720  
1                     ed sheeran    3562543890    3909458734  
2                  lewis capaldi    2887241814    3427498835  
3                    tones and i    2864791672    3071214106  
4          post malone, swae lee    2808096550    3358704125  
5            drake, wizkid, kyla    27

**Analysis 6: Stream Growth**

Finally, we will calculate the change in streams from 2023 to 2024, and put the percentage in a new column of our DataFrame.

In [697]:
# Calculate the change in streams
df_comparison['Growth In Streams'] = df_comparison['Streams_2024'] - df_comparison['Streams_2023']

# Calculate the percentage change and round to 2 decimal places
df_comparison['Growth Percentage'] = ((df_comparison['Streams_2024'] - df_comparison['Streams_2023']) / df_comparison['Streams_2023']) * 100
df_comparison['Growth Percentage'] = df_comparison['Growth Percentage'].round(2)

print('Top-10 Stream Count Growth from 2023-2024:')

print(df_comparison)

Top-10 Stream Count Growth from 2023-2024:
                                            Track  \
0                                 blinding lights   
1                                    shape of you   
2                               someone you loved   
3                                    dance monkey   
4   sunflower - spider-man: into the spider-verse   
5                                       one dance   
6                       stay (with justin bieber)   
10                                       believer   
11                                         closer   
12                                        starboy   

                          Artist  Streams_2023  Streams_2024  \
0                     the weeknd    3703895074    4281468720   
1                     ed sheeran    3562543890    3909458734   
2                  lewis capaldi    2887241814    3427498835   
3                    tones and i    2864791672    3071214106   
4          post malone, swae lee    2808096550    335

**CONCLUSION**

We have accomplished all of our goals originally set at the beginning of our project.

1. Idenitify the most-common musical attributes of all 2023 Spotify songs.

        2023 Musical Attributes:
        Musical Attribute | Most Common Value Among 2023 Spotify Songs
        BPM                                     120
        Key                                     C#
        Mode                                    Major
2. Count the occurances of those most-common attributes within the entire spotify-2023.csv file, as well as within only the top-10 most streamed songs of 2023. 

        2023 Attribute Value Count:
        Musical Attribute | Most Common Value | Attribute Value Count | Attribute Value Percentage
        BPM                     120                   34                        3.97
        Key                     C#                    120                       14.00
        Mode                    Major                 474                       55.31


        Attribute Value Count in the top-10 2023 Spotify songs:
        Musical Attribute | Most Common Value | Top 10 Count | Top 10 Percentage
        BPM                     120               0               0.0
        Key                     C#                5               50.0
        Mode                    Major             7               70.0


3. Track the top-10 songs' stream count from 2023-2024.

        Top-10 Stream Growth 2023-2024:
        Growth In Streams | Growth Percentage  
        577573646              15.59  
        346914844               9.74  
        540257021              18.71  
        206422434               7.21  
        550607575              19.61  
        478281716              17.62  
        441756427              16.57  
        412186629              15.89  
        289312697              11.17  
        725732720              28.29 

In [698]:
#DataFrames to csv for Visualizations:
df_common_attributes.to_csv('common_attributes.csv', index=False)
df_attribute_count.to_csv('attribute_count.csv', index=False)
df_attribute_count_top10.to_csv('attribute_count_top10.csv', index=False)
df_comparison.to_csv('comparison.csv', index=False)

In [699]:
pd.show_versions()


INSTALLED VERSIONS
------------------
commit                : d9cdd2ee5a58015ef6f4d15c7226110c9aab8140
python                : 3.12.2.final.0
python-bits           : 64
OS                    : Darwin
OS-release            : 23.5.0
Version               : Darwin Kernel Version 23.5.0: Wed May  1 20:09:52 PDT 2024; root:xnu-10063.121.3~5/RELEASE_X86_64
machine               : x86_64
processor             : i386
byteorder             : little
LC_ALL                : None
LANG                  : None
LOCALE                : None.UTF-8

pandas                : 2.2.2
numpy                 : 1.26.4
pytz                  : 2024.1
dateutil              : 2.9.0.post0
setuptools            : None
pip                   : 24.2
Cython                : None
pytest                : None
hypothesis            : None
sphinx                : None
blosc                 : None
feather               : None
xlsxwriter            : None
lxml.etree            : None
html5lib              : None
pymysql       